# Create instances of Data from Abacus simulations

In [1]:
import numpy as np

from abacusnbody.data.compaso_halo_catalog import CompaSOHaloCatalog
from densitysplit import catalog_data

## Output directory

In [11]:
output_dir = '/feynman/work/dphp/mp270220/data/'

### Get halo catalog

In [15]:
# 1 Gpc/h
path_to_sim1 = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_highbase_c000_ph100/halos/z1.175'

# 2 Gpc/h
path_to_sim2_z1 = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_base_c000_ph003/halos/z1.175'
path_to_sim2_z2 = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_base_c000_ph000/halos/z0.800'

path_to_sim = [path_to_sim1,
               path_to_sim2_z1,
               path_to_sim2_z2]

catalog_names = ['AbacusSummit_1Gpc_z1.175',
                 'AbacusSummit_2Gpc_z1.175_ph003',
                 'AbacusSummit_2Gpc_z0.800']

sim_z = [1.175, 
         1.175, 
         0.800]

sim_boxsizes = [1000, 
                2000, 
                2000]

for i in range(1, 2):
    halo_catalog = CompaSOHaloCatalog(path_to_sim[i], cleaned=True, fields=['id', 'x_L2com','v_L2com','N'])
    name = catalog_names[i]
    boxsize = sim_boxsizes[i]
    z = sim_z[i]
    positions = halo_catalog.halos['x_L2com']
    velocities = halo_catalog.halos['v_L2com']
    weights = halo_catalog.halos['N']
    
    # Transpose arrays
    positions_reshaped = np.array(positions.T, dtype='f8')
    weights_reshaped = np.array(weights.T, dtype='f8')
    velocities_reshaped = np.array(velocities.T, dtype='f8')

    # Create Data instance
    data_catalog = catalog_data.Data(positions_reshaped, z, boxsize, boxcenter=0, name=name, 
                                     weights=weights_reshaped, velocities=velocities_reshaped,
                                     mass_cut=500)
    
    # Save Data instance
    data_catalog.save(output_dir+name)

In [18]:
output_dir = '/feynman/work/dphp/mp270220/outputs/correlation_functions/'
data_catalog.shift_boxcenter(-data_catalog.offset)
edges = (np.linspace(0., 150., 51), np.linspace(-1, 1, 201))
los = 'x'
from pycorr import TwoPointCorrelationFunction
result = TwoPointCorrelationFunction('smu', edges,
                                     data_positions1=data_catalog.positions,
                                     boxsize=data_catalog.boxsize,
                                     engine='corrfunc', nthreads=128,
                                     los = los)
result.save(output_dir+catalog.name+'_2PCF')

RuntimeError: RuntimeError occurred

In [10]:
result.save(output_dir+data_catalog.name+'_nocut_2PCF')

In [7]:
result

In [27]:
data_catalog.size / data_catalog.boxsize**3

0.048493062

### Get all particles

In [49]:
from pathlib import Path
from abacusnbody.data import read_abacus
import astropy.table

path_to_sim = '/feynman/scratch/dphp/ar264273/Abacus/AbacusSummit_base_c000_ph000/halos/z0.800/'

allp = []
for fn in Path(path_to_sim).glob('*_rv_*/*.asdf'):
    batch = read_abacus.read_asdf(fn, load=['pos', 'vel'])
    batch_size = len(batch)
    sample = np.random.randint(0, batch_size, np.int(batch_size/10))
    allp += [batch[sample]]
allp = astropy.table.vstack(allp)

/tmp/ipykernel_3710552/1205069590.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample = np.random.randint(0, batch_size, np.int(batch_size/10))


In [50]:
output_dir = '/feynman/work/dphp/mp270220/data/'
name = 'AbacusSummit_2Gpc_z0.800_downsampled_particles'
boxsize = 2000
z = 0.8
positions = allp['pos']
velocities = allp['vel']

# Transpose arrays
positions_reshaped = np.array(positions.T, dtype='f8')
velocities_reshaped = np.array(velocities.T, dtype='f8')

# Create Data instance
data_catalog = catalog_data.Data(positions_reshaped, z, boxsize, boxcenter=0, name=name, 
                                 weights=None, velocities=velocities_reshaped,
                                 mass_cut=None)

In [51]:
nbar = data_catalog.size / 2000**3

In [52]:
nbar

0.123835594375

In [53]:
# Save Data instance
#data_catalog.save(output_dir+name+'_nbar{:.3f}'.format(nbar))

In [2]:
def compute_cic_density(data, smoothing_radius, cellsize=None, use_rsd=False, use_weights=False):
    boxsize = data.boxsize
    offset = data.boxcenter - data.boxsize/2.
    
    if use_rsd and data.positions_rsd is not None:
        positions = data.positions_rsd
    else:
        positions = data.positions
        
    if use_weights and data.weights is not None:
        weights = data.weights
        norm = np.sum(weights) * (4/3 * np.pi * smoothing_radius**3) / boxsize**3
    else:
        weights = None
        norm = data.size * (4/3 * np.pi * smoothing_radius**3) / boxsize**3

    if cellsize is None:
        cellsize = smoothing_radius * 2
    else:
        if cellsize < 2 * smoothing_radius:
            print("Cellsize must be bigger than twice the smoothing radius.")
    
    def compute_density_mesh(pos):
        indices_in_grid = ((pos - offset) / cellsize + 0.5).astype('i4')
        grid_pos = indices_in_grid * cellsize + offset
        dist_to_nearest_node = np.sum((grid_pos - pos)**2, axis=0)**0.5
        mask_particles = dist_to_nearest_node < smoothing_radius

        nmesh = np.int32(boxsize / cellsize)
        mask_particles &= np.all((indices_in_grid > 0) & (indices_in_grid < nmesh), axis=0)
        mesh = np.zeros((nmesh - 1,)*3, dtype='f8')
        np.add.at(mesh, tuple(indices_in_grid[:, mask_particles] - 1), weights[mask_particles] if use_weights else 1.)
        return mesh

    data_mesh = compute_density_mesh(positions)
    mesh = data_mesh / norm - 1

    return mesh

In [3]:
cellsize = 20
R = 10

#data_name = 'AbacusSummit_2Gpc_z0.800'
data_name = 'AbacusSummit_2Gpc_z0.800_downsampled_particles_nbar0.124'

density_cic = compute_cic_density(data_catalog, R, cellsize=cellsize)

output_dir = '/feynman/work/dphp/mp270220/outputs/'
#np.save(output_dir+data_name+'_density_cic_R{:02d}Mpc'.format(R), density_cic)
np.save(output_dir+data_name+'_density_contrast_cic_R{:02d}Mpc'.format(R), density_cic)

NameError: name 'data_catalog' is not defined